In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from utilities.misc import *
from utilities.templates import * 
from utilities.circuit_database import CirqTranslater
from utilities.simplifier import Simplifier

In [2]:
translator = CirqTranslater(n_qubits = 3)
simplifier = Simplifier(translator)

In [3]:
uu = pd.read_csv("testing/data/rul6")
circuit_db = uu.drop(columns=["Unnamed: 0"])
circuit, circuit_db = translator.give_circuit(circuit_db)

In [4]:
circuit

┌─────────┐
(0, 0): ───Rz(th_0)───Rx(th_3)───Rz(th_4)──────────────────────────────────────────────────────@────Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                               │                                        │
(0, 1): ───Rx(th_1)───H─────────────────────X────────────────X───H────────────────X────Rz(th_8)┼────────────────────────────────────────┼───────────────
                                            │                │                    │            │                                        │
(0, 2): ───Rx(th_2)───Rz(th_5)───H──────────@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────X────Rx(th_12)───Rz(th_13)───Rx(th_14)───X───Rz(th_15)───
                                                                                      └─────────┘

In [5]:
gates_on_qubit, on_qubit_order = simplifier.get_positional_dbs(circuit, circuit_db)

In [12]:
def rule_6(simplifier, simplified_db, on_qubit_order, gates_on_qubit):

    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            if simplification is True:
                break

            ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]

            ## if i have a rotation and then a CNOT
            if (check_rot(ind_gate, translator) is True) and (check_cnot(ind_gate_p1, translator) is True):
                type_0 = type_get(ind_gate, translator)
                control, target = translator.indexed_cnots[str(ind_gate_p1)]

                this_qubit = q
                other_qubits = [control, target]
                other_qubits.remove(q)
                other_qubit = other_qubits[0]

                ### now it happens two interesting things: type0 == 0 AND q == control
                ### or type_0 == 1 AND q == target  then swap orders

                if ((type_0 == 0) and (q == control)) or ((type_1 == 1) and (q == target)):
                    if len(on_qubit_order[q]) <2:
                        simplification=False
                    else:

                        simplification = True

                        ###now we swap the order in which we apply the rotation and the CNOT.
                        index_rot = on_qubit_order[q][order_gate_on_qubit]
                        info_rot = simplified_db.loc[index_rot].copy()
                        simplified_db = simplified_db.drop(labels=[index_rot],axis=0)#
                        #info_cnot = simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1]].copy()
                        #simplified_db.loc[on_qubit_order[q][order_gate_on_qubit]] = info_cnot
                        simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1 ] + 0.1] = info_rot
                        simplified_db = simplified_db.sort_index().reset_index(drop=True)
                        #simplified_db = sort_by_symbol(simplified_db)
                        break
    return simplification, simplified_db                                    

In [13]:
simplified_db = circuit_db.copy()

In [14]:
s, simplified_db = rule_6(simplifier, simplified_db, on_qubit_order, gates_on_qubit)

In [15]:
translator.give_circuit(circuit_db)

(                                                                                      ┌─────────┐
(0, 0): ───Rz(th_0)───Rx(th_3)───Rz(th_4)──────────────────────────────────────────────────────@────Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                               │                                        │
(0, 1): ───Rx(th_1)───H─────────────────────X────────────────X───H────────────────X────Rz(th_8)┼────────────────────────────────────────┼───────────────
                                            │                │                    │            │                                        │
(0, 2): ───Rx(th_2)───Rz(th_5)───H──────────@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────X────Rx(th_12)───Rz(th_13)───Rx(th_14)───X───Rz(th_15)───
                                                                                      └─────────┘,
     ind symbol  param_value  trainable  block_id channel_param
 0 

In [18]:
sisi = simplified_db.copy()

In [22]:
usisi = translator.give_circuit(sisi, unresolved=False)[0].unitary()

In [23]:
usisi_sorted = translator.give_circuit(sort_by_symbol(sisi), unresolved=False)[0].unitary()

In [28]:
sisi

,ind,symbol,param_value,trainable,block_id,channel_param
0,6,th_0,-15.195752,True,0,NaN
1,10,th_1,9.424775,True,0,NaN
2,11,th_2,-3.702385,True,0,NaN
3,9,th_3,25.173183,True,0,NaN
4,8,th_5,-1.570797,True,0,NaN
5,16,None,NaN,False,1,NaN
6,17,None,NaN,False,1,NaN
7,5,None,NaN,False,1,NaN
8,14,th_6,1.000000,False,1,True
9,5,None,NaN,False,1,NaN


In [35]:
translator.give_circuit(sisi)

(                                                                               ┌─────────┐
(0, 0): ───Rz(th_0)───Rx(th_3)──────────────────────────────────────────────────────────@────Rz(th_4)────Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                        │                                                    │
(0, 1): ───Rx(th_1)───H──────────────X────────────────X───H────────────────X────Rz(th_8)┼────────────────────────────────────────────────────┼───────────────
                                     │                │                    │            │                                                    │
(0, 2): ───Rx(th_2)───Rz(th_5)───H───@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────X────Rx(th_12)───Rz(th_13)───Rx(th_14)───────────────X───Rz(th_15)───
                                                                               └─────────┘,
     ind symbol  param_value  trainable  block_id channe

we need to work on the symbols now !!!

In [37]:
sisi.to_csv("testing/data/sorting")

In [34]:
translator.give_circuit(sort_by_symbol(sisi))

((0, 0): ───Rz(th_0)───Rx(th_3)───Rz(th_4)────────────────────────────────────────────────────@───Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                             │                                       │
(0, 1): ───Rx(th_1)───H──────────X─────────────────────X───H────────────────X───Rz(th_8)─────┼───────────────────────────────────────┼───────────────
                                 │                     │                    │                │                                       │
(0, 2): ───Rx(th_2)───H──────────@──────────Rz(th_5)───@───H───Ry(0.318π)───@───Ry(0.318π)───X───Rx(th_12)───Rz(th_13)───Rx(th_14)───X───Rz(th_15)───,
     ind symbol  param_value  trainable  block_id channel_param
 0     6   th_0   -15.195752       True         0           NaN
 1    10   th_1     9.424775       True         0           NaN
 2    11   th_2    -3.702385       True         0           NaN
 3     9   th_3    25.

In [31]:
translator.give_circuit(sisi)[0]

┌─────────┐
(0, 0): ───Rz(th_0)───Rx(th_3)──────────────────────────────────────────────────────────@────Rz(th_4)────Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                        │                                                    │
(0, 1): ───Rx(th_1)───H──────────────X────────────────X───H────────────────X────Rz(th_8)┼────────────────────────────────────────────────────┼───────────────
                                     │                │                    │            │                                                    │
(0, 2): ───Rx(th_2)───Rz(th_5)───H───@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────X────Rx(th_12)───Rz(th_13)───Rx(th_14)───────────────X───Rz(th_15)───
                                                                               └─────────┘

In [32]:
translator.give_circuit(sort_by_symbol(sisi))[0]

(0, 0): ───Rz(th_0)───Rx(th_3)───Rz(th_4)────────────────────────────────────────────────────@───Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                             │                                       │
(0, 1): ───Rx(th_1)───H──────────X─────────────────────X───H────────────────X───Rz(th_8)─────┼───────────────────────────────────────┼───────────────
                                 │                     │                    │                │                                       │
(0, 2): ───Rx(th_2)───H──────────@──────────Rz(th_5)───@───H───Ry(0.318π)───@───Ry(0.318π)───X───Rx(th_12)───Rz(th_13)───Rx(th_14)───X───Rz(th_15)───

In [ ]:
simplified_db = rule_6(simplifier, simplified_db, on_qubit_order, gates_on_qubit)

In [ ]:
simplified_db = circuit_db.copy()

In [ ]:
cc, simplified_db =  translator.give_circuit(simplified_db)

In [79]:
cc

┌─────────┐
(0, 0): ───Rz(th_0)───Rx(th_3)───Rz(th_4)──────────────────────────────────────────────────────@────Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                               │                                        │
(0, 1): ───Rx(th_1)───H─────────────────────X────────────────X───H────────────────X────Rz(th_8)┼────────────────────────────────────────┼───────────────
                                            │                │                    │            │                                        │
(0, 2): ───Rx(th_2)───Rz(th_5)───H──────────@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────X────Rx(th_12)───Rz(th_13)───Rx(th_14)───X───Rz(th_15)───
                                                                                      └─────────┘

In [61]:
translator.give_circuit(simplified_db)

(                                                                                      ┌─────────┐
(0, 0): ───Rz(th_0)───Rx(th_3)───Rz(th_4)──────────────────────────────────────────────────────@────Rz(th_9)────Rx(th_10)───Rz(th_11)───@───────────────
                                                                                               │                                        │
(0, 1): ───Rx(th_1)───H─────────────────────X────────────────X───H────────────────X────Rz(th_8)┼────────────────────────────────────────┼───────────────
                                            │                │                    │            │                                        │
(0, 2): ───Rx(th_2)───Rz(th_5)───H──────────@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────X────Rx(th_12)───Rz(th_13)───Rx(th_14)───X───Rz(th_15)───
                                                                                      └─────────┘,
     ind symbol  param_value  trainable  block_id channel_param
 0 

In [46]:
circuit_db

,ind,symbol,param_value,trainable,block_id,channel_param
0,6,th_0,-15.195752,True,0,NaN
1,10,th_1,9.424775,True,0,NaN
2,11,th_2,-3.702385,True,0,NaN
3,9,th_3,25.173183,True,0,NaN
4,6,th_4,43.483597,True,0,NaN
5,8,th_5,-1.570797,True,0,NaN
6,16,None,NaN,False,1,NaN
7,17,None,NaN,False,1,NaN
8,5,None,NaN,False,1,NaN
9,14,th_6,1.000000,False,1,True


In [45]:
simplified_db

,ind,symbol,param_value,trainable,block_id,channel_param
0,6,th_0,-15.195752,True,0,NaN
1,10,th_1,9.424775,True,0,NaN
2,11,th_2,-3.702385,True,0,NaN
3,9,th_3,25.173183,True,0,NaN
4,1,None,NaN,True,2,NaN
5,8,th_5,-1.570797,True,0,NaN
6,16,None,NaN,False,1,NaN
7,17,None,NaN,False,1,NaN
8,5,None,NaN,False,1,NaN
9,14,th_6,1.000000,False,1,True


In [18]:
ss = circuit_db.copy()

In [20]:
ss.loc[1] = ss.loc[0]

In [24]:
aa = ss.loc[5].copy()
bb = ss.loc[0].copy()

In [23]:
ss.loc[4] = aa

In [25]:
ss.loc[5] = bb

In [26]:
ss

,ind,symbol,param_value,trainable,block_id,channel_param
0,6,th_0,-15.195752,True,0,NaN
1,6,th_0,-15.195752,True,0,NaN
2,11,th_2,-3.702385,True,0,NaN
3,9,th_3,25.173183,True,0,NaN
4,8,th_5,-1.570797,True,0,NaN
5,6,th_0,-15.195752,True,0,NaN
6,16,None,NaN,False,1,NaN
7,17,None,NaN,False,1,NaN
8,5,None,NaN,False,1,NaN
9,14,th_6,1.000000,False,1,True


In [15]:
check_rot(gates_on_qubit[0][-1], translator)

False

In [16]:
check_cnot(gates_on_qubit[0][-1], translator)

True